<a href="https://githubtocolab.com/rramosp/ai4eng_example_project/blob/main/02%20-%20Preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
#bibliotecas de machine learning
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate, ShuffleSplit

from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn import utils
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,KFold,GroupKFold
import lightgbm as lgb
import gc


import warnings
warnings.filterwarnings('ignore')


## 1. Lectura y concatenación de los Dataset

In [2]:
#cargar los datos
building_met = pd.read_csv("building_metadata.csv")
train = pd.read_csv("train.csv")
weather_train = pd.read_csv("weather_train.csv")

In [3]:
building_met

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN
...,...,...,...,...,...,...
1444,15,1444,Entertainment/public assembly,19619,1914.0,NaN
1445,15,1445,Education,4298,NaN,NaN
1446,15,1446,Entertainment/public assembly,11265,1997.0,NaN
1447,15,1447,Lodging/residential,29775,2001.0,NaN


In [4]:
train

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.000
1,1,0,2016-01-01 00:00:00,0.000
2,2,0,2016-01-01 00:00:00,0.000
3,3,0,2016-01-01 00:00:00,0.000
4,4,0,2016-01-01 00:00:00,0.000
...,...,...,...,...
20216095,1444,0,2016-12-31 23:00:00,8.750
20216096,1445,0,2016-12-31 23:00:00,4.825
20216097,1446,0,2016-12-31 23:00:00,0.000
20216098,1447,0,2016-12-31 23:00:00,159.575


In [5]:
weather_train

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6
...,...,...,...,...,...,...,...,...,...
139768,15,2016-12-31 19:00:00,3.0,NaN,-8.0,NaN,NaN,180.0,5.7
139769,15,2016-12-31 20:00:00,2.8,2.0,-8.9,NaN,1007.4,180.0,7.7
139770,15,2016-12-31 21:00:00,2.8,NaN,-7.2,NaN,1007.5,180.0,5.1
139771,15,2016-12-31 22:00:00,2.2,NaN,-6.7,NaN,1008.0,170.0,4.6


In [6]:
#Concatenamos los datos de train con building_met
train = train.merge(building_met,on = 'building_id',how = 'left')

In [8]:
#Contatenamos los datos con el dataset weather_train
train = train.merge(weather_train, on=['site_id','timestamp'], how='left')

In [9]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


In [10]:
#Función para obtener información de la variable timstamp como mes, dia de la semana, etc
def breakdown_timestamp(dataframe):
    
    dataframe['timestamp']= pd.to_datetime(dataframe['timestamp'])

    dataframe['hour']= np.uint8(dataframe['timestamp'].dt.hour)
    
    dataframe['day']= np.uint16(dataframe['timestamp'].dt.day)
    dataframe['dayofweek']= np.uint8(dataframe['timestamp'].dt.dayofweek)
    dataframe['dayofyear']= np.uint16(dataframe['timestamp'].dt.dayofyear)

    dataframe['month']= np.uint8(dataframe['timestamp'].dt.month)

    dataframe['year']= np.uint16(dataframe['timestamp'].dt.year)
    
    return dataframe

In [11]:
train = breakdown_timestamp(train)

In [12]:
train

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,dayofweek,dayofyear,month,year
0,0,0,2016-01-01 00:00:00,0.000,0,Education,7432,2008.0,NaN,25.0,...,NaN,1019.7,0.0,0.0,0,1,4,1,1,2016
1,1,0,2016-01-01 00:00:00,0.000,0,Education,2720,2004.0,NaN,25.0,...,NaN,1019.7,0.0,0.0,0,1,4,1,1,2016
2,2,0,2016-01-01 00:00:00,0.000,0,Education,5376,1991.0,NaN,25.0,...,NaN,1019.7,0.0,0.0,0,1,4,1,1,2016
3,3,0,2016-01-01 00:00:00,0.000,0,Education,23685,2002.0,NaN,25.0,...,NaN,1019.7,0.0,0.0,0,1,4,1,1,2016
4,4,0,2016-01-01 00:00:00,0.000,0,Education,116607,1975.0,NaN,25.0,...,NaN,1019.7,0.0,0.0,0,1,4,1,1,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20216095,1444,0,2016-12-31 23:00:00,8.750,15,Entertainment/public assembly,19619,1914.0,NaN,1.7,...,-1.0,1008.5,180.0,8.8,23,31,5,366,12,2016
20216096,1445,0,2016-12-31 23:00:00,4.825,15,Education,4298,NaN,NaN,1.7,...,-1.0,1008.5,180.0,8.8,23,31,5,366,12,2016
20216097,1446,0,2016-12-31 23:00:00,0.000,15,Entertainment/public assembly,11265,1997.0,NaN,1.7,...,-1.0,1008.5,180.0,8.8,23,31,5,366,12,2016
20216098,1447,0,2016-12-31 23:00:00,159.575,15,Lodging/residential,29775,2001.0,NaN,1.7,...,-1.0,1008.5,180.0,8.8,23,31,5,366,12,2016


## 4. Tratamiento de datos

#### 4.1. Remoción de las lecturas cero de la variable objetivo

Como se se mencionó anteriormente al analizar la variable objetivo, se observó que existes muchas lecturas de valor cero para la variable objetivo (meter_reading). Debido a esto se opta por eliminar las filas que contengan un valor cero para la variable objetivo ya que esto indica que no se tiene una disponibilidad de la medición y estos no podrían utilizarse para entrenar.

In [31]:
meter_readings_zero = list(train[train.meter_reading == 0].index) #lista con los indices de los valores que tienen lecturas 0
train.drop(meter_readings_zero, axis = 0, inplace = True) #eliminar filas con lecturas cero en la variable objetivo
print('Nuevo tamaño de los datos: ',train.shape)

Nuevo tamaño de los datos:  (18342124, 22)


#### 4.2. Eliminación de las columnas con muchos datos faltantes

Del análisis exploratorio de las variables, se encontró que existen variables que contienen gran cantidad de datos faltantes. En este caso se optará por eliminar las columnas en las que los datos faltantes representen el 50% o más de la totalidad de los datos. Siguiendo este criterio se eliminan las variables "floor_count", "year_built".

In [32]:
criterio = len(train) * 0.5 #criterio para eliminar la columna (50% de las filas que se tienen)
train.dropna(axis=1, thresh = criterio, inplace = True) #eliminación de las columnas con 50% o más de datos faltantes
print('New Shape of Train Data:',train.shape)

New Shape of Train Data: (18342124, 20)


#### 4.4. Relleno de datos faltantes
Anteriormente se optó por eliminar las columnas que poseian el 50% o más en datos en datos faltantes, sin embargo, aun quedan variables con datos faltantes en el dataset. Para estas columnas se rellenaran los valores faltantes al reemplazarlos por la media de los valores con los que se cuenta para cada columna.

In [33]:
train['cloud_coverage'].fillna(train.cloud_coverage.median(), inplace=True)
train['sea_level_pressure'].fillna(train.sea_level_pressure.median(), inplace=True)
train['precip_depth_1_hr'].fillna(train.precip_depth_1_hr.median(), inplace=True)
train['wind_direction'].fillna(train.wind_direction.median(), inplace=True)
train['wind_speed'].fillna(train.wind_speed.median(), inplace=True)
train['dew_temperature'].fillna(train.dew_temperature.median(), inplace=True)
train['air_temperature'].fillna(train.air_temperature.median(), inplace=True)

In [34]:
train.isnull().sum()

building_id           0
meter                 0
timestamp             0
meter_reading         0
site_id               0
primary_use           0
square_feet           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
hour                  0
day                   0
dayofweek             0
dayofyear             0
month                 0
year                  0
dtype: int64

#### 4.4. Adición de la estación y variable para saber si esta de día o de noche

Para este análisis se encuentra interesante el hecho de saber de que estación del año es el cada dato, ya que normalmente en sistemas que ver con calentamiento de agua, o enfriadores de agua para sistemas de aire acondicionado, o sistemas de vapor, la estación del año puede influir en la carga que se impone en los sistemas aumentando o disminuyendo su consumo. También se puede tener que ciertas aplicaciones sean mas usadas en una estación que otra, como por ejemplo el uso del agua caliente en verano, el cual podría reducirse en esta estación. Además de esto la carga también tiene variaciones en el día o noche, y los consumos pueden presentar una variación dependiendo de esto.

In [35]:
# Adicionamos la estación del año, lo cual puede mejorar el análisis

train['season'] = train['timestamp'].apply(lambda x: 'Spring' if x.month==3 or x.month==4 or x.month==5 else 
                                                  'Summer' if x.month==6 or x.month==7 or x.month==8 else 
                                                  'Autumn' if x.month==9 or x.month==10 or x.month==11 else 
                                                  'Winter')

#Adicionamos una variable para saber si es de día, 1 es de día, 0 es de noche
train['isDayTime'] = train['timestamp'].apply(lambda x: 1 if x.hour >=6 and x.hour <=18 else 0)

In [36]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,...,wind_direction,wind_speed,hour,day,dayofweek,dayofyear,month,year,season,isDayTime
45,46,0,2016-01-01,3.993413,0,Retail,9.110078,25.0,6.0,20.0,...,0.0,0.0,0,1,4,1,1,2016,Winter,0
72,74,0,2016-01-01,3.784219,0,Parking,12.867830,25.0,6.0,20.0,...,0.0,0.0,0,1,4,1,1,2016,Winter,0
91,93,0,2016-01-01,3.978196,0,Office,10.415443,25.0,6.0,20.0,...,0.0,0.0,0,1,4,1,1,2016,Winter,0
103,105,0,2016-01-01,3.190624,1,Education,10.832181,3.8,0.0,2.4,...,240.0,3.1,0,1,4,1,1,2016,Winter,0
104,106,0,2016-01-01,0.318163,1,Education,8.589514,3.8,0.0,2.4,...,240.0,3.1,0,1,4,1,1,2016,Winter,0


#### 4.5. Transformación de las variables categoricas
Las variables categoricas, como lo son en este caso "primary_use" y "season" pueden ser utiles a la hora de realizar el análisis, sin embargo, no pueden usadas en la forma categorica, por lo que se deben convertir en variables numericas que si podamos utilzar para entrenar un modelo.

In [37]:
var_categoricas = ['primary_use', 'season']
encoder = preprocessing.LabelEncoder()

for i in var_categoricas:
    
    train[i] = encoder.fit_transform(train[i])
    
print (train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18342124 entries, 45 to 20216099
Data columns (total 22 columns):
 #   Column              Dtype         
---  ------              -----         
 0   building_id         int64         
 1   meter               int64         
 2   timestamp           datetime64[ns]
 3   meter_reading       float64       
 4   site_id             int64         
 5   primary_use         int32         
 6   square_feet         float64       
 7   air_temperature     float64       
 8   cloud_coverage      float64       
 9   dew_temperature     float64       
 10  precip_depth_1_hr   float64       
 11  sea_level_pressure  float64       
 12  wind_direction      float64       
 13  wind_speed          float64       
 14  hour                uint8         
 15  day                 uint16        
 16  dayofweek           uint8         
 17  dayofyear           uint16        
 18  month               uint8         
 19  year                uint16        
 20 

In [38]:
train = train.drop(['timestamp'],axis=1) #eliminamos la columna timestamp ya que no se utilizará

In [39]:
train.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,...,wind_direction,wind_speed,hour,day,dayofweek,dayofyear,month,year,season,isDayTime
45,46,0,3.993413,0,11,9.110078,25.0,6.0,20.0,0.0,...,0.0,0.0,0,1,4,1,1,2016,3,0
72,74,0,3.784219,0,8,12.867830,25.0,6.0,20.0,0.0,...,0.0,0.0,0,1,4,1,1,2016,3,0
91,93,0,3.978196,0,6,10.415443,25.0,6.0,20.0,0.0,...,0.0,0.0,0,1,4,1,1,2016,3,0
103,105,0,3.190624,1,0,10.832181,3.8,0.0,2.4,0.0,...,240.0,3.1,0,1,4,1,1,2016,3,0
104,106,0,0.318163,1,0,8.589514,3.8,0.0,2.4,0.0,...,240.0,3.1,0,1,4,1,1,2016,3,0
